In [128]:
import girderformindlogger
import girder_client as gc
import pandas as pd
from girderformindlogger.utility.response import isodatetime
import numpy as np

In [179]:
girder = gc.GirderClient(host="dev.mindlogger.org")

# Create a new user

In [180]:
def testCreateUser():
    randomUser = np.random.randint(1000000)
    firstName = 'test'
    lastName = 'user'
    # NOTE: girder makes login and email lowercase!!!
    login = 'testuser{}'.format(randomUser)
    email = 'testemail{}@testemail.com'.format(randomUser)
    password = 'password'
    createUser = girder.post('user', parameters=dict(firstName=firstName,
                                        lastName=lastName,
                                        login=login,
                                        email=email,
                                        password=password
                                       )
               )
    # assert 'authToken' in createUser.keys(), 'user has no token, {}'.format(createUser)
    assert createUser['email'] == email, 'email does not match, {} {}'.format(createUser['email'], email)
    assert createUser['firstName'] == firstName, 'firstName does not match'
    assert createUser['lastName'] == lastName, 'lastName does not match'
    assert createUser['login'] == login, 'login does not match, {} {}'.format(createUser['login'], login)
    assert createUser['admin'] == False, 'user is an admin'
    assert createUser['_modelType'] == 'user', 'model is not user'
    assert createUser['public'] == False, 'user is public!'

    return createUser

In [182]:
new_user = testCreateUser()
girder.authenticate(username=new_user['login'], password='password')

{'_accessLevel': 2,
 '_id': '5d828a24a2a2577370c218ed',
 '_modelType': 'user',
 'admin': False,
 'created': '2019-09-18T19:48:52.478000+00:00',
 'creatorId': '5d828a24a2a2577370c218ed',
 'email': 'testemail360036@testemail.com',
 'emailVerified': False,
 'firstName': 'test',
 'lastName': 'user',
 'login': 'testuser360036',
 'otp': False,
 'public': False,
 'size': 0,
 'status': 'enabled'}

# Get applets when the user doesn't have any

In [187]:
def getAppletsNewUser(new_user):
    girder.authenticate(username=new_user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == 0, 'a new user should have an empty list of applets. we get {}'.format(appletList)
    return 1

In [188]:
getAppletsNewUser(new_user)

1

# Add an applet

In [189]:
new_user = testCreateUser()
girder.authenticate(username=new_user['login'], password='password')

# TODO: create an activity-set that JUST for testing.
# make sure it has all the weird qualities that can break


# for now, lets do the mindlogger demo
activitySetUrl = 'http://repronim.org/schema-standardization/activity-sets/mindlogger-demo/mindlogger-demo_schema'

ar = girder.post('applet', parameters=dict(activity_set_url = activitySetUrl, name='testActivitySet'))



In [190]:
ar

{'_id': '5d828ab9a2a2577370c218ef',
 'access': {'groups': [],
  'users': [{'flags': [], 'id': '5d828ab7a2a2577370c218ee', 'level': 2}]},
 'baseParentId': '5c6aec8e9c7ea5529ee58db2',
 'baseParentType': 'collection',
 'created': '2019-09-18T19:51:21.702575+00:00',
 'creatorId': '5d828ab7a2a2577370c218ee',
 'description': '',
 'lowerName': 'testactivityset',
 'meta': {'activitySet': {'_id': 'activitySet/None', 'url': None},
  'applet': {}},
 'name': 'testActivitySet',
 'parentCollection': 'collection',
 'parentId': '5c6aec8e9c7ea5529ee58db2',
 'public': True,
 'roles': {'editor': {'groups': [{'id': '5d828abaa2a2577370c218f1'}],
   'users': []},
  'manager': {'groups': [{'id': '5d828abaa2a2577370c218f2'}], 'users': []},
  'reviewer': {'groups': [{'id': '5d828abaa2a2577370c218f3', 'subject': None}],
   'users': []},
  'user': {'groups': [{'id': '5d828ab9a2a2577370c218f0', 'subject': None}],
   'users': []}},
 'size': 0,
 'updated': '2019-09-18T19:51:21.796657+00:00'}

In [191]:
girder.get('applet/{}'.format(ar['_id']))

HttpError: HTTP error 500: GET https://dev.mindlogger.org:443/api/v1/applet/5d828ab9a2a2577370c218ef
Response text: {"message": "An unexpected error occurred on the server.", "type": "internal", "uid": "5dea3208-9104-470b-a4e3-f59d100b46a1"}

# Delete a user

In [192]:
def testDeleteUser(new_user):

    girder.authenticate(username=new_user['login'], password='password')

    deleteUser = girder.delete('user/{}'.format(new_user['_id']))

    assert deleteUser['message'] == 'Deleted user {}.'.format(new_user['login']), "{} does not equal {}".format(deleteUser['message'], 'Deleted user {}'.format(new_user['login']))
    
    return 1

In [193]:
testDeleteUser(new_user)

1